# import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/MLMA/project/Code Continuous Data')

In [ ]:
%load_ext autoreload
%autoreload 2
from stroke_probability_map import *

In [ ]:
%load_ext autoreload
%autoreload 2
from stroke_probability_map_1 import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import numpy as np
import os
import math
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

import pandas as pd

from scipy import signal

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils import data as Data

import torch
from torch.utils.data import Dataset, DataLoader
import torch.optim.lr_scheduler as lr_scheduler

from tqdm import tqdm

import pickle

In [ ]:
!pip install python-Levenshtein

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 91.5 MB/s eta 0:00:00


In [ ]:
from Levenshtein import ratio

# File and Path

File name:   
EOG_00x_xx_xxx.csv:   
 00x: participant's ID.    
 xx:  index of the recording time.    
 xxx: index of the corresponding eye movements.    
      - e.g., xxx is the index of a training word.   

In [ ]:
result_path = '/content/drive/MyDrive/Colab Notebooks/MLMA/project/Code Continuous Data/Result'

In [ ]:
data_path = '/content/drive/MyDrive/Colab Notebooks/MLMA/project/Data/continuous'

In [ ]:
result_path_isolated = '/content/drive/MyDrive/Colab Notebooks/MLMA/project/Code Isolated Data/Result'

# Dataset and Dataloader

In [ ]:
stroke_labels = np.load('/content/drive/MyDrive/Colab Notebooks/MLMA/project/Code Continuous Data/stroke_labels.npy')

In [ ]:
stroke_labels_2 = np.load('/content/drive/MyDrive/Colab Notebooks/MLMA/project/Code Continuous Data/stroke_labels_2.npy')

In [ ]:
stroke_labels_16 = np.append(stroke_labels, np.ones((150, 4))*12, axis=1)

In [ ]:
stroke_labels_3 = np.load('/content/drive/MyDrive/Colab Notebooks/MLMA/project/Code Continuous Data/stroke_labels_3.npy')

In [ ]:
stroke_labels_4 = np.load('/content/drive/MyDrive/Colab Notebooks/MLMA/project/Code Continuous Data/stroke_labels_4.npy')

In [ ]:
class MyDataset(Dataset):
  def __init__(self, ID_list, R_time_list, model_list, flatten, model_type, s_num, root_dir='/content/drive/MyDrive/Colab Notebooks/MLMA/project/Data/continuous/00'):
    self.data = []
    self.stroke_probability_map = []
    self.label = []
    if s_num == 0:
      s_labels = stroke_labels
    elif s_num == 1:
      s_labels = stroke_labels_2
    elif s_num == 2:
      s_labels = stroke_labels_16
    elif s_num == 3:
      s_labels = stroke_labels_3
    elif s_num == 4:
      s_labels = stroke_labels_4

    for id_idx in range(len(ID_list)):
      file_list = os.listdir(root_dir + str(ID_list[id_idx]) +'/training/')
      for f_idx in range(len(file_list)):
        if int(file_list[f_idx][8:10]) in R_time_list:
          data = pd.read_csv(root_dir + str(ID_list[id_idx]) +'/training/' + file_list[f_idx],  header=None).to_numpy()
          #print(data.shape)
          data = signal.resample(data, 1024, axis=0)
          #print(data.shape)
          s = stroke_probability_map_1(data, model_list[id_idx], flatten, model_type)
          s = torch.from_numpy(s)
          self.stroke_probability_map.append(s)
          word_idx = int(file_list[f_idx][-7:-4])-1
          self.label.append(torch.from_numpy(s_labels[word_idx]))
          data = (data - data.mean(axis=0, keepdims=True))/np.std(data,axis=0, keepdims=True)
          data = torch.from_numpy(data)
          self.data.append(data)

  def __getitem__(self, index) :
    return self.data[index].float(), self.stroke_probability_map[index].float(), self.label[index].type(torch.LongTensor)
  def __len__(self):
    return len(self.data)

# Model

In [ ]:
# 2+lstm
class My_CNN_2_lstm_stroke(nn.Module):
  def __init__(self, N=16):
    super(My_CNN_2_lstm_stroke, self).__init__()
    self.conv1 = nn.Conv1d(in_channels=2, out_channels=16, kernel_size=9, padding=5)
    nn.init.xavier_uniform_(self.conv1.weight)
    nn.init.constant_(self.conv1.bias, 0.0)
    self.mp1 = nn.MaxPool1d(kernel_size=4)
    self.relu1 = nn.LeakyReLU()

    self.conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=5, padding=3)
    nn.init.xavier_uniform_(self.conv2.weight)
    nn.init.constant_(self.conv2.bias, 0.0)
    self.mp2 = nn.MaxPool1d(kernel_size=2)
    self.relu2 = nn.LeakyReLU()

    self.conv3 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, padding=2)
    nn.init.xavier_uniform_(self.conv3.weight)
    nn.init.constant_(self.conv3.bias, 0.0)
    self.mp3 = nn.MaxPool1d(kernel_size=2)
    self.relu3 = nn.LeakyReLU()

    self.conv4 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, padding=2)
    nn.init.xavier_uniform_(self.conv4.weight)
    nn.init.constant_(self.conv4.bias, 0.0)
    self.amp =  nn.AdaptiveMaxPool1d(N)
    self.sigmoid = nn.Sigmoid()

    self.lstm = nn.LSTM(input_size=int(3*12+128), hidden_size=25, num_layers=1, batch_first=True, bidirectional=False, dropout=0.5)
    for name, param in self.lstm.named_parameters():
      if name.startswith("weight"):
        nn.init.xavier_normal_(param)
      else:
        nn.init.zeros_(param)

    self.fc1 = nn.Linear(16, 12)
    nn.init.xavier_uniform_(self.fc1.weight)
    nn.init.constant_(self.fc1.bias, 0.0)
    self.relu4 = nn.LeakyReLU()
    self.dropout = nn.Dropout(p=0.5)

    self.fc2 = nn.Linear(25, 13)
    nn.init.xavier_uniform_(self.fc2.weight)
    nn.init.constant_(self.fc2.bias, 0.0)

  def forward(self, x, probability):
    x = torch.transpose(x, -1, -2)

    x = self.relu1(self.mp1(self.conv1(x)))
    x = self.relu2(self.mp2(self.conv2(x)))
    x = self.relu3(self.mp3(self.conv3(x)))
    x = self.conv4(x)
    x = self.sigmoid(self.amp(x))
    x = torch.transpose(x, -1, -2)
    x = torch.cat((x, probability), dim=-1)

    x, (_, _) = self.lstm(x)
    x = torch.transpose(x, -1, -2)
    x = self.relu4(self.fc1(x))
    x = self.dropout(x)
    x = torch.transpose(x, -1, -2)
    x = self.fc2(x)

    x = torch.transpose(x, -1, -2)
    return x

In [ ]:
# 5_lstm
class My_CNN_5_lstm_stroke(nn.Module):
  def __init__(self, N=16):
    super(My_CNN_5_lstm_stroke, self).__init__()
    self.conv1 = nn.Conv1d(in_channels=2, out_channels=64, kernel_size=9, padding=5)
    nn.init.xavier_uniform_(self.conv1.weight)
    nn.init.constant_(self.conv1.bias, 0.0)
    self.mp1 = nn.MaxPool1d(kernel_size=4)
    self.relu1 = nn.LeakyReLU()

    self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=5, padding=3)
    nn.init.xavier_uniform_(self.conv2.weight)
    nn.init.constant_(self.conv2.bias, 0.0)
    self.mp2 = nn.MaxPool1d(kernel_size=2)
    self.relu2 = nn.LeakyReLU()

    self.conv3 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, padding=2)
    nn.init.xavier_uniform_(self.conv3.weight)
    nn.init.constant_(self.conv3.bias, 0.0)
    self.mp3 = nn.MaxPool1d(kernel_size=2)
    self.relu3 = nn.LeakyReLU()

    self.conv4 = nn.Conv1d(in_channels=256, out_channels=512, kernel_size=3, padding=2)
    nn.init.xavier_uniform_(self.conv4.weight)
    nn.init.constant_(self.conv4.bias, 0.0)
    self.amp =  nn.AdaptiveMaxPool1d(N)
    self.sigmoid = nn.Sigmoid()
    

    self.lstm = nn.LSTM(input_size=int(3*12+512), hidden_size=50, num_layers=1, batch_first=True, bidirectional=False, dropout=0.5)
    for name, param in self.lstm.named_parameters():
      if name.startswith("weight"):
        nn.init.xavier_normal_(param)
      else:
        nn.init.zeros_(param)

    self.fc1 = nn.Linear(16, 16)
    nn.init.xavier_uniform_(self.fc1.weight)
    nn.init.constant_(self.fc1.bias, 0.0)
    self.relu4 = nn.LeakyReLU()
    self.dropout = nn.Dropout(p=0.5)

    self.fc2 = nn.Linear(50, 13)
    nn.init.xavier_uniform_(self.fc2.weight)
    nn.init.constant_(self.fc2.bias, 0.0)

  def forward(self, x, probability):
    x = torch.transpose(x, -1, -2)

    x = self.relu1(self.mp1(self.conv1(x)))
    x = self.relu2(self.mp2(self.conv2(x)))
    x = self.relu3(self.mp3(self.conv3(x)))
    x = self.conv4(x)
    x = self.sigmoid(self.amp(x))
    x = torch.transpose(x, -1, -2)
    x = torch.cat((x, probability), dim=-1)

    x, (_, _) = self.lstm(x)
    x = torch.transpose(x, -1, -2)
    x = self.relu4(self.fc1(x))
    x = self.dropout(x)
    x = torch.transpose(x, -1, -2)
    x = self.fc2(x)

    x = torch.transpose(x, -1, -2)

    return x

In [ ]:
# 4_lstm
class My_CNN_4_lstm_stroke(nn.Module):
  def __init__(self, N=16):
    super(My_CNN_4_lstm_stroke, self).__init__()
    self.conv1 = nn.Conv1d(in_channels=2, out_channels=64, kernel_size=9, padding=5)
    nn.init.xavier_uniform_(self.conv1.weight)
    nn.init.constant_(self.conv1.bias, 0.0)
    self.mp1 = nn.MaxPool1d(kernel_size=4)
    self.relu1 = nn.LeakyReLU()

    self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=5, padding=3)
    nn.init.xavier_uniform_(self.conv2.weight)
    nn.init.constant_(self.conv2.bias, 0.0)
    self.mp2 = nn.MaxPool1d(kernel_size=2)
    self.relu2 = nn.LeakyReLU()

    self.conv3 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, padding=2)
    nn.init.xavier_uniform_(self.conv3.weight)
    nn.init.constant_(self.conv3.bias, 0.0)
    self.mp3 = nn.MaxPool1d(kernel_size=2)
    self.relu3 = nn.LeakyReLU()

    self.conv4 = nn.Conv1d(in_channels=256, out_channels=256, kernel_size=3, padding=2)
    nn.init.xavier_uniform_(self.conv4.weight)
    nn.init.constant_(self.conv4.bias, 0.0)
    self.amp =  nn.AdaptiveMaxPool1d(N)
    self.sigmoid = nn.Sigmoid()
    

    self.lstm = nn.LSTM(input_size=int(3*12+256), hidden_size=50, num_layers=1, batch_first=True, bidirectional=False, dropout=0.5)
    for name, param in self.lstm.named_parameters():
      if name.startswith("weight"):
        nn.init.xavier_normal_(param)
      else:
        nn.init.zeros_(param)

    self.fc1 = nn.Linear(16, 16)
    nn.init.xavier_uniform_(self.fc1.weight)
    nn.init.constant_(self.fc1.bias, 0.0)
    self.relu4 = nn.LeakyReLU()
    self.dropout = nn.Dropout(p=0.5)

    self.fc2 = nn.Linear(50, 13)
    nn.init.xavier_uniform_(self.fc2.weight)
    nn.init.constant_(self.fc2.bias, 0.0)

  def forward(self, x, probability):
    x = torch.transpose(x, -1, -2)

    x = self.relu1(self.mp1(self.conv1(x)))
    x = self.relu2(self.mp2(self.conv2(x)))
    x = self.relu3(self.mp3(self.conv3(x)))
    x = self.conv4(x)
    x = self.sigmoid(self.amp(x))
    x = torch.transpose(x, -1, -2)
    x = torch.cat((x, probability), dim=-1)

    x, (_, _) = self.lstm(x)
    x = torch.transpose(x, -1, -2)
    x = self.relu4(self.fc1(x))
    x = self.dropout(x)
    x = torch.transpose(x, -1, -2)
    x = self.fc2(x)

    x = torch.transpose(x, -1, -2)

    return x

In [ ]:
# 3_lstm
class My_CNN_3_lstm_stroke(nn.Module):
  def __init__(self, N=16):
    super(My_CNN_3_lstm_stroke, self).__init__()
    self.conv1 = nn.Conv1d(in_channels=2, out_channels=32, kernel_size=9, padding=5)
    nn.init.xavier_uniform_(self.conv1.weight)
    nn.init.constant_(self.conv1.bias, 0.0)
    self.mp1 = nn.MaxPool1d(kernel_size=4)
    self.relu1 = nn.LeakyReLU()

    self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=5, padding=3)
    nn.init.xavier_uniform_(self.conv2.weight)
    nn.init.constant_(self.conv2.bias, 0.0)
    self.mp2 = nn.MaxPool1d(kernel_size=2)
    self.relu2 = nn.LeakyReLU()

    self.conv3 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, padding=2)
    nn.init.xavier_uniform_(self.conv3.weight)
    nn.init.constant_(self.conv3.bias, 0.0)
    self.mp3 = nn.MaxPool1d(kernel_size=2)
    self.relu3 = nn.LeakyReLU()

    self.conv4 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, padding=2)
    nn.init.xavier_uniform_(self.conv4.weight)
    nn.init.constant_(self.conv4.bias, 0.0)
    self.amp =  nn.AdaptiveMaxPool1d(N)
    self.sigmoid = nn.Sigmoid()
    

    self.lstm = nn.LSTM(input_size=int(3*12+256), hidden_size=25, num_layers=1, batch_first=True, bidirectional=False, dropout=0.5)
    for name, param in self.lstm.named_parameters():
      if name.startswith("weight"):
        nn.init.xavier_normal_(param)
      else:
        nn.init.zeros_(param)

    self.fc1 = nn.Linear(16, 12)
    nn.init.xavier_uniform_(self.fc1.weight)
    nn.init.constant_(self.fc1.bias, 0.0)
    self.relu4 = nn.LeakyReLU()
    self.dropout = nn.Dropout(p=0.5)

    self.fc2 = nn.Linear(25, 13)
    nn.init.xavier_uniform_(self.fc2.weight)
    nn.init.constant_(self.fc2.bias, 0.0)

  def forward(self, x, probability):
    x = torch.transpose(x, -1, -2)

    x = self.relu1(self.mp1(self.conv1(x)))
    x = self.relu2(self.mp2(self.conv2(x)))
    x = self.relu3(self.mp3(self.conv3(x)))
    x = self.conv4(x)
    x = self.sigmoid(self.amp(x))
    x = torch.transpose(x, -1, -2)
    x = torch.cat((x, probability), dim=-1)

    x, (_, _) = self.lstm(x)
    x = torch.transpose(x, -1, -2)
    x = self.relu4(self.fc1(x))
    x = self.dropout(x)
    x = torch.transpose(x, -1, -2)
    x = self.fc2(x)

    x = torch.transpose(x, -1, -2)

    return x

cross entropy:

input N,C,...

# Some Functions

In [ ]:
def Train_model(model,epochs,optmizer,criterion, train_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.train()
    train_loss = []
    x = 0
    for batch_idx, (data, probability, label) in enumerate(train_loader):
        data = data.to(device)
        label = label.to(device)
        probability = probability.to(device)
        optmizer.zero_grad()  
        output = model(data, probability)
        # print(type(output), type(label))
        loss = criterion(output, label)
        loss.backward()
        optmizer.step() 
        train_loss.append(loss.item()) 
        #if(batch_idx+1)%3 == 0:
            #print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.\
                #format(epochs, batch_idx * len(data), len(train_loader.dataset), 100. * batch_idx / len(train_loader), loss.item()))
    return np.mean(train_loss), model

In [ ]:
def rle(x):
  return_list = [x[0]]
  x = np.append(x, np.ones((1,))*-1, axis=0)
  for i in range(1, x.shape[0]-1):
    if x[i] != x[i-1]:
      return_list.append(x[i])
  return return_list

In [ ]:
def score_list(y_true, y_pred):
  return accuracy_score(y_true, y_pred), \
  f1_score(y_true, y_pred, average='micro', zero_division=0), f1_score(y_true, y_pred, average='macro', zero_division=0), \
  precision_score(y_true, y_pred, average='micro', zero_division=0), precision_score(y_true, y_pred, average='macro', zero_division=0), \
  recall_score(y_true, y_pred, average='micro', zero_division=0), recall_score(y_true, y_pred, average='macro', zero_division=0)

In [ ]:
def test(model,epochs,criterion, dataloader, T_T='Test'):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    test_loss = 0
    c = 0
    y_pred_prob = np.zeros((0, 13, 16))
    y_true = np.zeros((0, 16))
    s_f = nn.Softmax(dim=1)
    with torch.no_grad():
        for data, probability, label in dataloader:
            data = data.to(device)
            label = label.to(device)
            probability = probability.to(device)
            output = model(data, probability)
            test_loss += criterion(output, label).item()
            output = s_f(output)
            y_pred_prob = np.append(y_pred_prob, output.cpu(), axis=0)
            y_true = np.append(y_true, label.cpu(), axis=0)
            c += 1
    test_loss /= c
    if epochs%50 == 0:
        print('EPOCH:{}    {} set: Average loss: {:.4f}'.format(epochs, T_T, test_loss))

    y_pred = y_pred_prob.argmax(axis=1)
    y_pred_gram = np.zeros(y_pred.shape)

    r = 0
    r_rle = 0
    r_gram = 0
    r_rle_gram = 0
    for i in range(y_pred.shape[0]):
      idx = np.where(y_pred[i] != 12)
      s1 = y_pred[i][idx]
      idx = np.where(y_true[i] != 12)
      s2 = y_true[i][idx]
      r+=ratio(s1, s2)
      s1 = rle(s1)
      s2 = rle(s2)
      r_rle+=ratio(s1, s2)

      s1_gram = [int(y_pred_prob[i, :-1, 0].argmax())]
      for j in range(1, 16):
        prob_ = np.multiply(y_pred_prob[i, :, j], gram_1[s1_gram[-1]])
        s1_gram.append(int(prob_.argmax()))
      s1_gram = np.array(s1_gram)
      y_pred_gram[i] = s1_gram
      idx = np.where(s1_gram != 12)
      s1 = s1_gram[idx]    
      idx = np.where(y_true[i] != 12)
      s2 = y_true[i][idx]
      r_gram += ratio(s1, s2)
      s1 = rle(s1)
      s2 = rle(s2)
      r_rle_gram += ratio(s1, s2)  

    r_rle = r_rle/y_pred.shape[0]
    r = r/y_pred.shape[0]
    r_rle_gram = r_rle_gram/y_pred.shape[0]
    r_gram = r_gram/y_pred.shape[0]

    y_pred = y_pred.reshape((-1))
    y_pred_gram = y_pred_gram.reshape((-1))
    y_true = y_true.reshape((-1))

    return_array = np.zeros((1+4+7*2))
    return_array[0] = test_loss
    return_array[1:5] = r, r_rle, r_gram, r_rle_gram
    return_array[5:(5+7)] = score_list(y_true, y_pred)
    return_array[(5+7):] = score_list(y_true, y_pred_gram)
    return return_array

In [ ]:
def test_with_gram(model,epochs,criterion, dataloader, T_T='Test'):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    test_loss = 0
    c = 0
    y_pred_prob = np.zeros((0, 13, 16))
    y_true = np.zeros((0, 16))
    s_f = nn.Softmax(dim=1)
    with torch.no_grad():
        for data, probability, label in dataloader:
            data = data.to(device)
            label = label.to(device)
            probability = probability.to(device)
            output = model(data, probability)
            test_loss += criterion(output, label).item()
            output = s_f(output)
            y_pred_prob = np.append(y_pred_prob, output.cpu(), axis=0)
            y_true = np.append(y_true, label.cpu(), axis=0)
            c += 1
    test_loss /= c
    if epochs%50 == 0:
        print('EPOCH:{}    {} set: Average loss: {:.4f}'.format(epochs, T_T, test_loss))

    y_pred = y_pred_prob.argmax(axis=1)

    r = np.zeros((4,))
    r_rle = np.zeros((4,))
    for i in range(y_pred.shape[0]):
      # no gram
      idx_12 = np.where(y_pred[i][1:] == 12)
      if len(idx_12[0]) > 0:
        idx = int(idx_12[0][0])
        s1 = y_pred[i][:(idx+1)]
      else:
        s1 = y_pred[i]
      idx = int(np.where(y_true[i]==12)[0][0])
      s2 = y_true[i][:idx]
      r[0]+=ratio(s1, s2)
      s1 = rle(s1)
      s2_rle = rle(s2)
      r_rle[0]+=ratio(s1, s2_rle)

      # gram 1
      s1_gram = [int(y_pred_prob[i, :-1, 0].argmax())]
      for j in range(1, 16):
        prob_ = np.multiply(y_pred_prob[i, :, j], gram_1[s1_gram[-1]])
        if int(prob_.argmax()) == 12:
          break
        s1_gram.append(int(prob_.argmax()))
      s1 = np.array(s1_gram)   
      r[1] += ratio(s1, s2)
      s1 = rle(s1)
      r_rle[1] += ratio(s1, s2_rle)  

      # gram 2
      s1_gram = [int(y_pred_prob[i, :-1, 0].argmax()), int(y_pred_prob[i, :-1, 1].argmax())]
      for j in range(2, 16):
        prob_ = np.multiply(y_pred_prob[i, :, j], gram_2[s1_gram[-2], s1_gram[-1]])
        if int(prob_.argmax()) == 12:
          break
        s1_gram.append(int(prob_.argmax()))
      s1 = np.array(s1_gram)   
      r[2] += ratio(s1, s2)
      s1 = rle(s1)
      r_rle[2] += ratio(s1, s2_rle) 

      # gram 3
      s1_gram = [int(y_pred_prob[i, :-1, 0].argmax()), int(y_pred_prob[i, :-1, 1].argmax()), int(y_pred_prob[i, :-1, 2].argmax())]
      for j in range(3, 16):
        prob_ = np.multiply(y_pred_prob[i, :, j], gram_3[s1_gram[-3], s1_gram[-2], s1_gram[-1]])
        if int(prob_.argmax()) == 12:
          break
        s1_gram.append(int(prob_.argmax()))
      s1 = np.array(s1_gram)   
      r[3] += ratio(s1, s2)
      s1 = rle(s1)
      r_rle[3] += ratio(s1, s2_rle)  

    r_rle = r_rle/y_pred.shape[0]
    r = r/y_pred.shape[0]
    return np.append(np.ones((1,))*test_loss, np.append(r, r_rle, axis=0), axis=0)

In [ ]:
def test(model,epochs,criterion, dataloader, T_T='Test'):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    test_loss = 0
    c = 0
    y_pred_prob = np.zeros((0, 13, 16))
    y_true = np.zeros((0, 16))
    s_f = nn.Softmax(dim=1)
    with torch.no_grad():
        for data, probability, label in dataloader:
            data = data.to(device)
            label = label.to(device)
            probability = probability.to(device)
            output = model(data, probability)
            test_loss += criterion(output, label).item()
            output = s_f(output)
            y_pred_prob = np.append(y_pred_prob, output.cpu(), axis=0)
            y_true = np.append(y_true, label.cpu(), axis=0)
            c += 1
    test_loss /= c
    if epochs%50 == 0:
        print('EPOCH:{}    {} set: Average loss: {:.4f}'.format(epochs, T_T, test_loss))

    y_pred = y_pred_prob.argmax(axis=1)
    y_pred_gram = np.zeros(y_pred.shape)

    r = 0
    r_rle = 0
    r_gram = 0
    r_rle_gram = 0
    for i in range(y_pred.shape[0]):
      idx = np.where(y_pred[i] != 12)
      s1 = y_pred[i][idx]
      idx = np.where(y_true[i] != 12)
      s2 = y_true[i][idx]
      r+=ratio(s1, s2)
      s1 = rle(s1)
      s2 = rle(s2)
      r_rle+=ratio(s1, s2)

      s1_gram = [int(y_pred_prob[i, :-1, 0].argmax())]
      for j in range(1, 16):
        prob_ = np.multiply(y_pred_prob[i, :, j], gram_1[s1_gram[-1]])
        s1_gram.append(int(prob_.argmax()))
      s1_gram = np.array(s1_gram)
      y_pred_gram[i] = s1_gram
      idx = np.where(s1_gram != 12)
      s1 = s1_gram[idx]    
      idx = np.where(y_true[i] != 12)
      s2 = y_true[i][idx]
      r_gram += ratio(s1, s2)
      s1 = rle(s1)
      s2 = rle(s2)
      r_rle_gram += ratio(s1, s2)  

    r_rle = r_rle/y_pred.shape[0]
    r = r/y_pred.shape[0]
    r_rle_gram = r_rle_gram/y_pred.shape[0]
    r_gram = r_gram/y_pred.shape[0]

    y_pred = y_pred.reshape((-1))
    y_pred_gram = y_pred_gram.reshape((-1))
    y_true = y_true.reshape((-1))

    return_array = np.zeros((1+4+7*2))
    return_array[0] = test_loss
    return_array[1:5] = r, r_rle, r_gram, r_rle_gram
    return_array[5:(5+7)] = score_list(y_true, y_pred)
    return_array[(5+7):] = score_list(y_true, y_pred_gram)
    return return_array

In [ ]:
class_num = np.zeros((13))
for i in range(0, 13):
    class_num[i] = np.sum(stroke_labels_16==i)
print(class_num)
weights = np.sum(class_num)/(class_num*13)
print(weights)

[ 132.   94.  167.  135.   25.  113.   29.   15.   71.   43.   34.   61.
 1481.]
[ 1.3986014   1.96399345  1.10548135  1.36752137  7.38461538  1.63376447
  6.36604775 12.30769231  2.60021668  4.29338104  5.42986425  3.02648172
  0.1246559 ]


In [ ]:
def main_function(total_epoch=100, dataloader_list=None, model_para=None, model_save=False, model_name=None, cross_val=True, weight_decay=1e-3):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    model = My_CNN_5_lstm_stroke().to(device)
        
    optmizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=weight_decay)  
    #optmizer = optim.SGD(model.parameters(), lr=1e-3, weight_decay=weight_decay) 
    scheduler = lr_scheduler.StepLR(optmizer, step_size=50, gamma=0.5, verbose=False) 
    weight=torch.tensor(weights).to(device).float()
    criterion = nn.CrossEntropyLoss(weight=weight).to(device)  
    
    train_loss = np.zeros((total_epoch))
    score = np.zeros((len(dataloader_list), 8+1, total_epoch)) # loss, ratio, accuracy, f1 score*2, precision*2, recall*2
    
    for i in tqdm(range(total_epoch)):
        train_loss[i], model = Train_model(model, i, optmizer, criterion, dataloader_list[0])
        scheduler.step()
        for j in range(len(dataloader_list)):
            if j == 0:
                T_T = 'Train'
            elif j == 1:
                T_T = 'Validation'
            else:
                T_T = 'Test'
            score[j, :, i] = test_with_gram(model, i, criterion, dataloader_list[j], T_T=T_T)
        #if np.argmin(score[1, 0, :(i+1)]) == i:
            #if model_save:
                #torch.save(model.state_dict(), model_name)     
    #if model_save: 
        #np.savez(model_name[:-4], train_loss=train_loss, score=score)
    if model_save:
      torch.save(model.state_dict(), model_name)
    if cross_val:
        epoch_best = np.argmin(score[1, 0, :])
        return score[:, :, epoch_best]
    else:
        return train_loss, score

# Leave one trial out

In [ ]:
BATCH_SIZE = 128
ID_list = [1,2,3,4,5,6]
model_name_list = ['mlp', 'SVC']
save_number = 0
s_num = 0

for m_idx in range(1):
  model_list = []
  for ID in range(1, 7):
    model_name = result_path_isolated + '/'+ model_name_list[m_idx] + '_sub00' + str(ID) + '.pck'
    model_stroke = pickle.load(open(model_name, "rb"))
    model_list.append(model_stroke)

  score = np.zeros((4, 3, 9))
  for fold_idx in range(1, 5):
    test_dataset = MyDataset(ID_list, R_time_list=[fold_idx], model_list=model_list, flatten=True, model_type=0, s_num=s_num)
    test_loader = Data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

    x = fold_idx+1 if fold_idx+1<=5 else 1
    val_dataset = MyDataset(ID_list, R_time_list=[x], model_list=model_list, flatten=True, model_type=0, s_num=s_num)
    val_loader = Data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

    R_time_list = []
    for i in range(1, 9):
      if i!=fold_idx and i!=x:
        R_time_list.append(i)
    train_dataset = MyDataset(ID_list, R_time_list=R_time_list, model_list=model_list, flatten=True, model_type=0, s_num=s_num)
    train_loader = Data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

      
    model_param=64
    score[fold_idx-1] = main_function(total_epoch=300, dataloader_list=[train_loader, val_loader, test_loader], \
                                  model_para=model_param, model_save=False, cross_val=True, weight_decay=1e-4)
    print('fold{} Train Loss{}, Train Acc{} ratio{}'.format(fold_idx, score[fold_idx-1, 0, 0], score[fold_idx-1, 0, 1], score[fold_idx-1, 0, 2]))
    print('fold{} Val Loss{}, Val Acc{} ratio{}'.format(fold_idx, score[fold_idx-1, 1, 0], score[fold_idx-1, 1, 1], score[fold_idx-1, 1, 2]))
    print('fold{} Test Loss{}, Test Acc{} ratio{}'.format(fold_idx, score[fold_idx-1, 2, 0], score[fold_idx-1, 2, 1], score[fold_idx-1, 2, 2]))
    torch.cuda.empty_cache()

  np.save(result_path + '/STROKE_d0420_score_' + str(save_number), score)
  save_number += 1

  x = np.mean(score, axis=0)
  print('Train', x[0])
  print('Val', x[1])
  print('Test', x[2])

/usr/local/lib/python3.9/dist-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
  0%|          | 0/300 [00:00<?, ?it/s]

EPOCH:0    Train set: Average loss: 2.4969
EPOCH:0    Validation set: Average loss: 2.4969


  0%|          | 1/300 [00:00<02:55,  1.71it/s]

EPOCH:0    Test set: Average loss: 2.4949


 17%|█▋        | 50/300 [00:28<02:22,  1.75it/s]

...

EPOCH:250    Train set: Average loss: 0.7608
EPOCH:250    Validation set: Average loss: 1.3843


 84%|████████▎ | 251/300 [02:30<00:29,  1.68it/s]

EPOCH:250    Test set: Average loss: 1.2240


100%|██████████| 300/300 [02:59<00:00,  1.67it/s]

fold4 Train Loss1.053864443844015, Train Acc0.5814108372494451 ratio0.6845868321771936
fold4 Val Loss1.248142696917057, Val Acc0.5521316409688093 ratio0.6505196733481812
fold4 Test Loss1.2399060726165771, Test Acc0.5386330424479997 ratio0.64293659621802
Train [0.89275197 0.62193701 0.71359813 0.71359813 0.6042101  0.71359813
 0.56201444 0.71359813 0.73137049]
Val [1.16988605 0.5724191  0.66877273 0.66877273 0.51889779 0.66877273
 0.48890727 0.66877273 0.61318907]
Test [1.15732414 0.57123512 0.66751656 0.66751656 0.51951108 0.66751656
 0.48926736 0.66751656 0.61420537]


In [ ]:
BATCH_SIZE = 128
ID_list = [1,2,3,4,5,6]
model_name_list = ['mlp', 'SVC']

save_number = 1
s_num = 2

for m_idx in range(1):
  model_list = []
  for ID in range(1, 7):
    model_name = result_path_isolated + '/'+ model_name_list[m_idx] + '_sub00' + str(ID) + '.pck'
    model_stroke = pickle.load(open(model_name, "rb"))
    model_list.append(model_stroke)

  score = np.zeros((4, 3, 9))
  for fold_idx in range(1, 5):
    test_dataset = MyDataset(ID_list, R_time_list=[fold_idx], model_list=model_list, flatten=True, model_type=0, s_num=s_num)
    test_loader = Data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

    x = fold_idx+1 if fold_idx+1<=5 else 1
    val_dataset = MyDataset(ID_list, R_time_list=[x], model_list=model_list, flatten=True, model_type=0, s_num=s_num)
    val_loader = Data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

    R_time_list = []
    for i in range(1, 9):
      if i!=fold_idx and i!=x:
        R_time_list.append(i)
    train_dataset = MyDataset(ID_list, R_time_list=R_time_list, model_list=model_list, flatten=True, model_type=0, s_num=s_num)
    train_loader = Data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

      
    model_param=64
    score[fold_idx-1] = main_function(total_epoch=300, dataloader_list=[train_loader, val_loader, test_loader], \
                                  model_para=model_param, model_save=False, cross_val=True, weight_decay=1e-4)
    print('fold{} Train Loss{}, Train ratio{} Acc{}'.format(fold_idx, score[fold_idx-1, 0, 0], score[fold_idx-1, 0, 1], score[fold_idx-1, 0, 2]))
    print('fold{} Val Loss{}, Val ratio{} Acc{}'.format(fold_idx, score[fold_idx-1, 1, 0], score[fold_idx-1, 1, 1], score[fold_idx-1, 1, 2]))
    print('fold{} Test Loss{}, Test ratio{} Acc{}'.format(fold_idx, score[fold_idx-1, 2, 0], score[fold_idx-1, 2, 1], score[fold_idx-1, 2, 2]))
    torch.cuda.empty_cache()

  np.save(result_path + '/STROKE_d0420_score_' + str(save_number), score)
  save_number += 1

  x = np.mean(score, axis=0)
  print('Train', x[0])
  print('Val', x[1])
  print('Test', x[2])

/usr/local/lib/python3.9/dist-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
  0%|          | 0/300 [00:00<?, ?it/s]

EPOCH:0    Train set: Average loss: 2.4550
EPOCH:0    Validation set: Average loss: 2.4553


  0%|          | 1/300 [00:00<03:10,  1.57it/s]

EPOCH:0    Test set: Average loss: 2.4606


 17%|█▋        | 50/300 [00:31<02:35,  1.61it/s]

...

EPOCH:250    Train set: Average loss: 0.2451
EPOCH:250    Validation set: Average loss: 1.0369


 84%|████████▎ | 251/300 [02:40<00:31,  1.56it/s]

EPOCH:250    Test set: Average loss: 0.9833


100%|██████████| 300/300 [03:12<00:00,  1.56it/s]

fold4 Train Loss0.402866546403278, Train Acc0.7806729885412748 ratio0.8683598028477546
fold4 Val Loss0.8726288825273514, Val Acc0.6731424747226127 ratio0.8030345211581291
fold4 Test Loss0.848213717341423, Test Acc0.6682150750481759 ratio0.8043659621802002
Train [0.50455341 0.72861312 0.84019482 0.84019482 0.74797939 0.84019482
 0.69359006 0.84019482 0.84837378]
Val [0.88930622 0.64505753 0.78930256 0.78930256 0.62626588 0.78930256
 0.58678697 0.78930256 0.69870364]
Test [0.88287721 0.64479215 0.79022343 0.79022343 0.6254642  0.79022343
 0.58532724 0.79022343 0.6986299 ]


In [ ]:
BATCH_SIZE = 128
ID_list = [1,2,3,4,5,6]
model_name_list = ['mlp', 'SVC']

save_number = 2
s_num = 2

for m_idx in range(1):
  model_list = []
  for ID in range(1, 7):
    model_name = result_path_isolated + '/'+ model_name_list[m_idx] + '_sub00' + str(ID) + '.pck'
    model_stroke = pickle.load(open(model_name, "rb"))
    model_list.append(model_stroke)

  score = np.zeros((4, 3, 10))
  for fold_idx in range(1, 5):
    test_dataset = MyDataset(ID_list, R_time_list=[fold_idx], model_list=model_list, flatten=True, model_type=0, s_num=s_num)
    test_loader = Data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

    x = fold_idx+1 if fold_idx+1<=5 else 1
    val_dataset = MyDataset(ID_list, R_time_list=[x], model_list=model_list, flatten=True, model_type=0, s_num=s_num)
    val_loader = Data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

    R_time_list = []
    for i in range(1, 9):
      if i!=fold_idx and i!=x:
        R_time_list.append(i)
    train_dataset = MyDataset(ID_list, R_time_list=R_time_list, model_list=model_list, flatten=True, model_type=0, s_num=s_num)
    train_loader = Data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

      
    model_param=64
    score[fold_idx-1] = main_function(total_epoch=300, dataloader_list=[train_loader, val_loader, test_loader], \
                                  model_para=model_param, model_save=False, cross_val=True, weight_decay=1e-4)
    print('fold{} Train Loss{}, Train ratio{} Acc{}'.format(fold_idx, score[fold_idx-1, 0, 0], score[fold_idx-1, 0, 1], score[fold_idx-1, 0, 2]))
    print('fold{} Val Loss{}, Val ratio{} Acc{}'.format(fold_idx, score[fold_idx-1, 1, 0], score[fold_idx-1, 1, 1], score[fold_idx-1, 1, 2]))
    print('fold{} Test Loss{}, Test ratio{} Acc{}'.format(fold_idx, score[fold_idx-1, 2, 0], score[fold_idx-1, 2, 1], score[fold_idx-1, 2, 2]))
    torch.cuda.empty_cache()

  np.save(result_path + '/STROKE_d0420_score_' + str(save_number), score)
  save_number += 1

  x = np.mean(score, axis=0)
  print('Train', x[0])
  print('Val', x[1])
  print('Test', x[2])

/usr/local/lib/python3.9/dist-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
  0%|          | 0/300 [00:00<?, ?it/s]

EPOCH:0    Train set: Average loss: 2.5294
EPOCH:0    Validation set: Average loss: 2.5287
EPOCH:0    Test set: Average loss: 2.5315


 17%|█▋        | 50/300 [00:47<03:33,  1.17it/s]

EPOCH:50    Train set: Average loss: 0.6925
EPOCH:50    Validation set: Average loss: 0.8860
EPOCH:50    Test set: Average loss: 0.8809


 33%|███▎      | 100/300 [01:30<02:49,  1.18it/s]

...

EPOCH:200    Train set: Average loss: 0.2932
EPOCH:200    Validation set: Average loss: 1.0881
EPOCH:200    Test set: Average loss: 1.0143


 83%|████████▎ | 250/300 [03:46<00:37,  1.32it/s]

EPOCH:250    Train set: Average loss: 0.2622
EPOCH:250    Validation set: Average loss: 1.1535
EPOCH:250    Test set: Average loss: 1.0428


100%|██████████| 300/300 [04:30<00:00,  1.11it/s]


fold4 Train Loss0.5268813791600141, Train ratio0.728257159655363 Acc0.7964331926064578
fold4 Val Loss0.9003293178975582, Val ratio0.6522450733761405 Acc0.7144644382354433
fold4 Test Loss0.884769681841135, Test ratio0.6515566141431847 Acc0.7140624933193749
Train [0.45511831 0.74525545 0.82186472 0.84146313 0.84146313 0.76000792
 0.84146313 0.69595952 0.84146313 0.86603326]
Val [0.85527348 0.66299783 0.73477843 0.78759824 0.78759824 0.63119274
 0.78759824 0.58489499 0.78759824 0.7067734 ]
Test [0.86468712 0.66016192 0.73426273 0.78584359 0.78584359 0.63331933
 0.78584359 0.58653934 0.78584359 0.71045577]


# d04-21

## character gram + character间隔1的label

In [ ]:
gram_1 = np.load('/content/drive/MyDrive/Colab Notebooks/MLMA/project/Code Continuous Data/gram_1_nopadding_include7_time3.npy')

In [ ]:
BATCH_SIZE = 128
ID_list = [1,2,3,4,5,6]
model_name_list = ['mlp', 'SVC']

save_number = 0
s_num = 3

for m_idx in range(1):
  model_list = []
  for ID in range(1, 7):
    model_name = result_path_isolated + '/'+ model_name_list[m_idx] + '_sub00' + str(ID) + '.pck'
    model_stroke = pickle.load(open(model_name, "rb"))
    model_list.append(model_stroke)

  score = np.zeros((4, 3, 1+4+7*2))
  for fold_idx in range(1, 5):
    test_dataset = MyDataset(ID_list, R_time_list=[fold_idx], model_list=model_list, flatten=True, model_type=0, s_num=s_num)
    test_loader = Data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

    x = fold_idx+1 if fold_idx+1<=5 else 1
    val_dataset = MyDataset(ID_list, R_time_list=[x], model_list=model_list, flatten=True, model_type=0, s_num=s_num)
    val_loader = Data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

    R_time_list = []
    for i in range(1, 9):
      if i!=fold_idx and i!=x:
        R_time_list.append(i)
    train_dataset = MyDataset(ID_list, R_time_list=R_time_list, model_list=model_list, flatten=True, model_type=0, s_num=s_num)
    train_loader = Data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

      
    model_param=64
    score[fold_idx-1] = main_function(total_epoch=200, dataloader_list=[train_loader, val_loader, test_loader], \
                                  model_para=model_param, model_save=False, cross_val=True, weight_decay=1e-4)
    print('Train', score[fold_idx-1, 0])
    print('Validation', score[fold_idx-1, 1])
    print('Test', score[fold_idx-1, 2])
    torch.cuda.empty_cache()

  np.save(result_path + '/STROKE_d0421_score_' + str(save_number), score)
  save_number += 1

  x = np.mean(score, axis=0)
  print('Train', x[0])
  print('Val', x[1])
  print('Test', x[2])

/usr/local/lib/python3.9/dist-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
  0%|          | 0/200 [00:00<?, ?it/s]

EPOCH:0    Train set: Average loss: 2.4757
EPOCH:0    Validation set: Average loss: 2.4748


  0%|          | 1/200 [00:01<04:11,  1.26s/it]

EPOCH:0    Test set: Average loss: 2.4725


 25%|██▌       | 50/200 [01:02<03:09,  1.26s/it]

...

EPOCH:150    Train set: Average loss: 0.3549
EPOCH:150    Validation set: Average loss: 0.8689


 76%|███████▌  | 151/200 [03:17<01:04,  1.31s/it]

EPOCH:150    Test set: Average loss: 0.8430


100%|██████████| 200/200 [04:20<00:00,  1.30s/it]

Train [0.41800578 0.70126033 0.85669743 0.75523306 0.84653305 0.7474215
 0.7474215  0.68392808 0.7474215  0.58657689 0.7474215  0.8753245
 0.7855513  0.7855513  0.71579466 0.7855513  0.64411762 0.7855513
 0.82582069]
Validation [0.81455456 0.63247757 0.77161614 0.6788851  0.75645964 0.69480791
 0.69480791 0.58199329 0.69480791 0.50461229 0.69480791 0.73227534
 0.72473552 0.72473552 0.5898177  0.72473552 0.54807839 0.72473552
 0.64669319]
Test [0.79881863 0.63656389 0.77534135 0.68397967 0.76435416 0.6958426
 0.6958426  0.58148573 0.6958426  0.50471588 0.6958426  0.72877001
 0.7253198  0.7253198  0.59107313 0.7253198  0.547052   0.7253198
 0.65258229]
Train [0.4410967  0.69841051 0.83310327 0.75723411 0.83250068 0.7404078
 0.7404078  0.68242015 0.7404078  0.58440628 0.7404078  0.8734702
 0.78571484 0.78571484 0.71982027 0.78571484 0.65344184 0.78571484
 0.81969663]
Val [0.8304875  0.6277053  0.7490118  0.68024694 0.74525219 0.68444416
 0.68444416 0.57753291 0.68444416 0.50194347 0.68444

## word gram

In [ ]:
t = np.load('/content/drive/MyDrive/Colab Notebooks/MLMA/project/Code Continuous Data/gram_word.npz')
gram_1 = t['gram_1_word']
gram_2 = t['gram_2_word']
gram_3 = t['gram_3_word']
del t

In [ ]:
BATCH_SIZE = 128
ID_list = [1,2,3,4,5,6]
model_name_list = ['mlp', 'SVC']

save_number = 1
s_num = 2

for m_idx in range(1):
  model_list = []
  for ID in range(1, 7):
    model_name = result_path_isolated + '/'+ model_name_list[m_idx] + '_sub00' + str(ID) + '.pck'
    model_stroke = pickle.load(open(model_name, "rb"))
    model_list.append(model_stroke)

  score = np.zeros((4, 3, 9))
  for fold_idx in range(1, 5):
    test_dataset = MyDataset(ID_list, R_time_list=[fold_idx], model_list=model_list, flatten=True, model_type=0, s_num=s_num)
    test_loader = Data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

    x = fold_idx+1 if fold_idx+1<=5 else 1
    val_dataset = MyDataset(ID_list, R_time_list=[x], model_list=model_list, flatten=True, model_type=0, s_num=s_num)
    val_loader = Data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

    R_time_list = []
    for i in range(1, 9):
      if i!=fold_idx and i!=x:
        R_time_list.append(i)
    train_dataset = MyDataset(ID_list, R_time_list=R_time_list, model_list=model_list, flatten=True, model_type=0, s_num=s_num)
    train_loader = Data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

      
    model_param=64
    score[fold_idx-1] = main_function(total_epoch=200, dataloader_list=[train_loader, val_loader, test_loader], \
                                  model_para=model_param, model_save=False, cross_val=True, weight_decay=1e-4)
    print('Train', score[fold_idx-1, 0])
    print('Validation', score[fold_idx-1, 1])
    print('Test', score[fold_idx-1, 2])
    torch.cuda.empty_cache()

  np.save(result_path + '/STROKE_d0421_score_' + str(save_number), score)
  save_number += 1

  x = np.mean(score, axis=0)
  print('Train', x[0])
  print('Val', x[1])
  print('Test', x[2])

/usr/local/lib/python3.9/dist-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
  0%|          | 0/200 [00:00<?, ?it/s]

EPOCH:0    Train set: Average loss: 2.5258
EPOCH:0    Validation set: Average loss: 2.5229
EPOCH:0    Test set: Average loss: 2.5263


 25%|██▌       | 50/200 [00:59<03:02,  1.22s/it]

EPOCH:50    Train set: Average loss: 0.8329
EPOCH:50    Validation set: Average loss: 1.0037


 26%|██▌       | 51/200 [01:01<03:02,  1.23s/it]

...

EPOCH:150    Train set: Average loss: 0.2017
EPOCH:150    Validation set: Average loss: 1.0289


 76%|███████▌  | 151/200 [03:06<01:00,  1.24s/it]

EPOCH:150    Test set: Average loss: 0.9864


100%|██████████| 200/200 [04:06<00:00,  1.23s/it]

Train [0.41534869 0.74415607 0.79276789 0.76880529 0.76518644 0.83984307
 0.85285284 0.84752802 0.85272329]
Validation [0.83155328 0.66832027 0.70349705 0.68435468 0.67922616 0.75664055
 0.75919743 0.75847897 0.76337295]
Test [0.82266323 0.65854519 0.70037885 0.67975462 0.66922224 0.75093723
 0.75856004 0.75866852 0.75877011]
Train [0.39468616 0.77110523 0.81002759 0.79226401 0.79488509 0.84355332
 0.85558637 0.85042858 0.86025445]
Val [0.82103087 0.67174122 0.7011487  0.68951134 0.6835609  0.74281169
 0.74782377 0.74805423 0.75133037]
Test [0.83137153 0.67497511 0.70693326 0.6967701  0.68853851 0.74749765
 0.75356446 0.75751419 0.756803  ]


In [ ]:
# 重新写了一下 test函数
BATCH_SIZE = 128
ID_list = [1,2,3,4,5,6]
model_name_list = ['mlp', 'SVC']

save_number = 3
s_num = 2

for m_idx in range(1):
  model_list = []
  for ID in range(1, 7):
    model_name = result_path_isolated + '/'+ model_name_list[m_idx] + '_sub00' + str(ID) + '.pck'
    model_stroke = pickle.load(open(model_name, "rb"))
    model_list.append(model_stroke)

  score = np.zeros((4, 3, 9))
  for fold_idx in range(1, 5):
    test_dataset = MyDataset(ID_list, R_time_list=[fold_idx], model_list=model_list, flatten=True, model_type=0, s_num=s_num)
    test_loader = Data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

    x = fold_idx+1 if fold_idx+1<=5 else 1
    val_dataset = MyDataset(ID_list, R_time_list=[x], model_list=model_list, flatten=True, model_type=0, s_num=s_num)
    val_loader = Data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

    R_time_list = []
    for i in range(1, 9):
      if i!=fold_idx and i!=x:
        R_time_list.append(i)
    train_dataset = MyDataset(ID_list, R_time_list=R_time_list, model_list=model_list, flatten=True, model_type=0, s_num=s_num)
    train_loader = Data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

      
    model_param=64
    score[fold_idx-1] = main_function(total_epoch=200, dataloader_list=[train_loader, val_loader, test_loader], \
                                  model_para=model_param, model_save=False, cross_val=True, weight_decay=1e-4)
    print('Train', score[fold_idx-1, 0])
    print('Validation', score[fold_idx-1, 1])
    print('Test', score[fold_idx-1, 2])
    torch.cuda.empty_cache()

  np.save(result_path + '/STROKE_d0421_score_' + str(save_number), score)
  save_number += 1

  x = np.mean(score, axis=0)
  print('Train', x[0])
  print('Val', x[1])
  print('Test', x[2])

/usr/local/lib/python3.9/dist-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
  0%|          | 0/200 [00:00<?, ?it/s]

EPOCH:0    Train set: Average loss: 2.4975
EPOCH:0    Validation set: Average loss: 2.4953
EPOCH:0    Test set: Average loss: 2.4974


 25%|██▌       | 50/200 [00:55<02:32,  1.02s/it]

EPOCH:50    Train set: Average loss: 0.7997
EPOCH:50    Validation set: Average loss: 0.9961
EPOCH:50    Test set: Average loss: 1.0164


 50%|█████     | 100/200 [01:45<01:41,  1.01s/it]

...

EPOCH:100    Train set: Average loss: 0.4188
EPOCH:100    Validation set: Average loss: 0.8532
EPOCH:100    Test set: Average loss: 0.9378


 75%|███████▌  | 150/200 [02:34<00:50,  1.02s/it]

EPOCH:150    Train set: Average loss: 0.2855
EPOCH:150    Validation set: Average loss: 0.9131
EPOCH:150    Test set: Average loss: 0.9638


100%|██████████| 200/200 [03:25<00:00,  1.03s/it]

Train [0.45691931 0.7566462  0.78652482 0.77583803 0.77937084 0.83485406
 0.83827779 0.84062001 0.85148113]
Validation [0.83535976 0.65529037 0.68626437 0.67651229 0.668752   0.73586753
 0.74305652 0.742642   0.74518833]
Test [0.90853204 0.65618034 0.68464696 0.67664609 0.66869172 0.7354599
 0.73583077 0.73746889 0.744329  ]
Train [0.41403263 0.77005931 0.80748319 0.79149037 0.79120821 0.84066273
 0.85313266 0.84800793 0.85574491]
Val [0.83749143 0.67316735 0.70331392 0.69149516 0.68571183 0.74215504
 0.75137385 0.74909002 0.75125968]
Test [0.85565311 0.6752075  0.70620178 0.69423504 0.68661761 0.74361276
 0.75239282 0.75036085 0.75122018]


## word gram + 12 -100 label

In [ ]:
t = np.load('/content/drive/MyDrive/Colab Notebooks/MLMA/project/Code Continuous Data/gram_word.npz')
gram_1 = t['gram_1_word']
gram_2 = t['gram_2_word']
gram_3 = t['gram_3_word']
del t

In [ ]:
BATCH_SIZE = 128
ID_list = [1,2,3,4,5,6]
model_name_list = ['mlp', 'SVC']

save_number = 2
s_num = 4

for m_idx in range(1):
  model_list = []
  for ID in range(1, 7):
    model_name = result_path_isolated + '/'+ model_name_list[m_idx] + '_sub00' + str(ID) + '.pck'
    model_stroke = pickle.load(open(model_name, "rb"))
    model_list.append(model_stroke)

  score = np.zeros((4, 3, 9))
  for fold_idx in range(1, 5):
    test_dataset = MyDataset(ID_list, R_time_list=[fold_idx], model_list=model_list, flatten=True, model_type=0, s_num=s_num)
    test_loader = Data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

    x = fold_idx+1 if fold_idx+1<=5 else 1
    val_dataset = MyDataset(ID_list, R_time_list=[x], model_list=model_list, flatten=True, model_type=0, s_num=s_num)
    val_loader = Data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

    R_time_list = []
    for i in range(1, 9):
      if i!=fold_idx and i!=x:
        R_time_list.append(i)
    train_dataset = MyDataset(ID_list, R_time_list=R_time_list, model_list=model_list, flatten=True, model_type=0, s_num=s_num)
    train_loader = Data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

      
    model_param=64
    score[fold_idx-1] = main_function(total_epoch=200, dataloader_list=[train_loader, val_loader, test_loader], \
                                  model_para=model_param, model_save=False, cross_val=True, weight_decay=1e-4)
    print('Train', score[fold_idx-1, 0])
    print('Validation', score[fold_idx-1, 1])
    print('Test', score[fold_idx-1, 2])
    torch.cuda.empty_cache()

  np.save(result_path + '/STROKE_d0421_score_' + str(save_number), score)
  save_number += 1

  x = np.mean(score, axis=0)
  print('Train', x[0])
  print('Val', x[1])
  print('Test', x[2])

/usr/local/lib/python3.9/dist-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
  0%|          | 0/200 [00:00<?, ?it/s]

EPOCH:0    Train set: Average loss: 2.4728
EPOCH:0    Validation set: Average loss: 2.4621
EPOCH:0    Test set: Average loss: 2.4628


 25%|██▌       | 50/200 [00:47<02:23,  1.05it/s]

EPOCH:50    Train set: Average loss: 0.8370
EPOCH:50    Validation set: Average loss: 1.0494
EPOCH:50    Test set: Average loss: 1.0578


 50%|█████     | 100/200 [01:35<01:35,  1.05it/s]

...

EPOCH:100    Train set: Average loss: 0.4901
EPOCH:100    Validation set: Average loss: 1.0729
EPOCH:100    Test set: Average loss: 1.0189


 75%|███████▌  | 150/200 [02:23<00:48,  1.04it/s]

EPOCH:150    Train set: Average loss: 0.3358
EPOCH:150    Validation set: Average loss: 1.1232
EPOCH:150    Test set: Average loss: 1.0702


100%|██████████| 200/200 [03:12<00:00,  1.04it/s]

Train [0.52470355 0.79198981 0.81164934 0.80872718 0.81874587 0.81150155
 0.82370798 0.82614435 0.83485297]
Validation [1.03905231 0.64681367 0.67168082 0.66251163 0.65945112 0.66485798
 0.68457141 0.68189442 0.67521524]
Test [1.02168284 0.65026955 0.67025619 0.66451907 0.66346068 0.66970769
 0.68446425 0.68230842 0.67881095]
Train [0.47369827 0.81029361 0.8327025  0.8300434  0.8354748  0.83372096
 0.84605195 0.84816048 0.85390257]
Val [0.92718951 0.68383452 0.70493439 0.70131631 0.69797474 0.71347837
 0.72416756 0.72701392 0.72437502]
Test [0.93587953 0.68471406 0.70319415 0.70136267 0.69899842 0.71392385
 0.72137564 0.72549419 0.72401032]


$P(Stroke_i==Class_j) = P(Class_{(i,j)}) * P(Class_j|Stroke_{i-1},..., Stroke_{i-N})$, where $N=1,2,3$

$P(Stroke_i==Class_j) = P(Class_{(i,j)}) * P(Class_j|Stroke_{i-1},..., Stroke_{i-N})$

# Train and predict on sentence

## Train Model

In [ ]:
ID_list = [1,2,3,4,5,6]
R_time_list = [1,2,3,4,5,6,7,8]
model_list = []
m_idx = 0
for ID in range(1, 7):
  model_name = result_path_isolated + '/'+ model_name_list[m_idx] + '_sub00' + str(ID) + '.pck'
  model_stroke = pickle.load(open(model_name, "rb"))
  model_list.append(model_stroke)
train_dataset = MyDataset(ID_list, R_time_list=R_time_list, model_list=model_list, flatten=True, model_type=0, s_num=2)
train_loader = Data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
model = My_CNN_5_lstm_stroke().to(device)

weight_decay=1e-4
total_epoch = 120

optmizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=weight_decay)  
scheduler = lr_scheduler.StepLR(optmizer, step_size=50, gamma=0.5, verbose=False) 
weight=torch.tensor(weights).to(device).float()
criterion = nn.CrossEntropyLoss(weight=weight).to(device)  
    
    
for i in tqdm(range(total_epoch)):
  _, model = Train_model(model, i, optmizer, criterion, train_loader)
  scheduler.step()

/usr/local/lib/python3.9/dist-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
100%|██████████| 120/120 [00:32<00:00,  3.67it/s]


In [ ]:
import scipy
# sentence
# path /content/drive/MyDrive/Colab Notebooks/MLMA/project/Data/continuous/001/test/EOG_001_01_001.csv
sentence_seg = torch.zeros((0, 1024, 2))
prob_seg = torch.zeros((0, 16, 36))
root_dir = '/content/drive/MyDrive/Colab Notebooks/MLMA/project/Data/continuous/00'

for ID in tqdm(range(1, 7)):
  model_name = result_path_isolated + '/mlp_sub00' + str(ID) + '.pck'
  model_sub = pickle.load(open(model_name, "rb"))
  for i in range(1, 26):
    sent_str = '0' + str(i) if i<10 else str(i)
    sentence_ = pd.read_csv(root_dir  + str(ID) + '/test/EOG_00' + str(ID) + '_01_0' + sent_str + '.csv').to_numpy()
    n_ = int(np.floor(sentence_.shape[0]/6))
    for j in range(6):
      seg_r = signal.resample(sentence_[(j*n_):((j+1)*n_)], 1024, axis=0)
      seg_r = torch.tensor(seg_r)
      sentence_seg = torch.cat((sentence_seg, seg_r.reshape((1, 1024, 2))), dim=0)

      s = stroke_probability_map_1(seg_r, model_sub, flatten=True, model_type=0)
      s = torch.from_numpy(s).reshape((1, 16, 36))

      prob_seg = torch.cat((prob_seg, s), dim=0)


sentence_seg = sentence_seg.float().to(device)
prob_seg = prob_seg.float().to(device)
print(sentence_seg.shape, prob_seg.shape)

100%|██████████| 6/6 [00:35<00:00,  5.98s/it]

torch.Size([900, 1024, 2]) torch.Size([900, 16, 36])


## Prediction

In [ ]:
y = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/MLMA/project/Data/continuous/50character_label.xlsx', header=None)
f2 = open("/content/drive/MyDrive/Colab Notebooks/MLMA/project/Data/continuous/test_sent.jp.txt","r")
lines = f2.readlines()

w = lines[0][7:-1].split(' ')
print(w[2][1], y.iloc[60, 0])
print(w[2][1] ==  y.iloc[60, 0])
y.loc[70] = [w[2][1], ' ', y.iloc[60, 2]]


w = lines[3][7:-1].split(' ')
print(w[5][3], y.iloc[59, 0])
print(w[5][3] ==  y.iloc[59, 0])
y.loc[71] = [w[5][3], ' ', y.iloc[59, 2]]

w = lines[1][7:-1].split(' ')
print(w[4][2], y.iloc[28, 0])
print(w[4][2] ==  y.iloc[28, 0])
y.loc[72] = [w[4][2], ' ', y.iloc[28, 2]]


charc_list = []
for i in range(y.shape[0]):
    cur_ = []
    temp = y.iloc[i, 2][1:-1].split(',')
    for k in range(len(temp)):
        cur_.append(int(temp[k]))
    charc_list.append(cur_)
            

sent_label = []
for idx in range(len(lines)):
    line3 = lines[idx]
    sent_label.append([])
    w = line3[7:-1].split(' ')
    for i in range(len(w)):
        for j in range(len(w[i])):
            # check through y
            find = False
            for k in range(y.shape[0]):
                if w[i][j] == y.iloc[k, 0]:
                    find = True
                    for m in range(len(charc_list[k])):
                        sent_label[-1].append(charc_list[k][m])
            if find == False:
                print(w[i][j], idx, i, j)

ョ ヨ
False
ュ ユ
False
ッ ツ
False


In [ ]:
output = model(sentence_seg, prob_seg)
output_label = output.cpu().argmax(dim=1).numpy()
output = output.cpu().detach().numpy()


result_0 = []
result_1 = []
result_2 = []
result_3 = []
for i in range(output.shape[0]):
  idx = np.where(output_label[i] != 12)[0]
  result_0.append(list(output_label[i][idx]))

  # 1
  r_ = [output[i, :-1, 0].argmax()]
  for j in range(1, 16):
    prob_ = np.multiply(output[i, :, j], gram_1[r_[-1]])
    p_next = int(prob_.argmax())
    if p_next == 12:
      break
    else:
      r_.append(p_next)
  result_1.append(r_)

  # 2
  r_ = [output[i, :-1, 0].argmax(), output[i, :-1, 1].argmax()]
  for j in range(2, 16):
    prob_ = np.multiply(output[i, :, j], gram_2[r_[-2], r_[-1]])
    p_next = int(prob_.argmax())
    if p_next == 12:
      break
    else:
      r_.append(p_next)
  result_2.append(r_)

  # 3
  r_ = [output[i, :-1, 0].argmax(), output[i, :-1, 1].argmax(), output[i, :-1, 2].argmax()]
  for j in range(2, 16):
    prob_ = np.multiply(output[i, :, j], gram_3[r_[-3], r_[-2], r_[-1]])
    p_next = int(prob_.argmax())
    if p_next == 12:
      break
    else:
      r_.append(p_next)
  result_3.append(r_)



In [ ]:
ratio_result = np.zeros((6, 4, 25))
ratio_rle_result = np.zeros((6, 4, 25))

for ID in range(1, 7):
  for s_idx in range(1, 26):
    b_idx = int((ID-1)*(6*25)+(s_idx-1) * 6)
    for i in range(4):
      if i == 0:
        r_c = result_0
      elif i == 1:
        r_c = result_1
      elif i == 2:
        r_c = result_2
      elif i == 3:
        r_c = result_3
      else:
        print('i error')
      result_c = []
      for cut_idx in range(6):
        result_c = result_c + r_c[b_idx+cut_idx]

      ratio_result[ID-1, i, s_idx-1] = ratio(result_c, sent_label[s_idx-1])

      result_c = rle(result_c)
      s_rle = rle(sent_label[s_idx-1])
      ratio_rle_result[ID-1, i, s_idx-1] = ratio(result_c, s_rle)

In [ ]:
print(np.mean(np.mean(ratio_result, axis=-1), axis=0))
print(np.mean(np.mean(ratio_rle_result, axis=-1), axis=0))

[0.53603532 0.54556513 0.53610813 0.50408278]
[0.5576614  0.55441889 0.55507792 0.55136716]


In [ ]:
np.savez('STROKE_d0421_score_ratio', ratio_result=ratio_result, ratio_rle_result=ratio_rle_result)